In [1]:
import os
# 아래는 우분투일때
# ipynb_path = os.getcwd()
# src_path = os.path.join(ipynb_path, '../src/')
# input_path = os.path.join("../input/")

ipynb_path = 'C:/Users/User/Documents/Wind_Power/notebooks'
src_path = 'C:/Users/User/Documents/Wind_Power/src'
input_path = "C:/Users/User/Documents/Wind_Power/input/"

In [2]:
!pip install windpowerlib

In [23]:

import warnings
import os
import sys

import scipy.stats as spst

sys.path.append(src_path)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import dask
import dask.dataframe as dd
from windpowerlib.wind_speed import logarithmic_profile
from utils import uv_to_wsd

In [24]:
os.getcwd()

'C:\\Users\\User'

In [25]:
power_2020 = pd.read_parquet(input_path + "dynamic_report_ewp02_2020_10min.parquet").rename({'Date/Time': 'dt', 'WTG.Name': 'turbine_id'}, axis=1)[:-3]
power_2021 = pd.read_parquet(input_path + "dynamic_report_ewp02_2021_10min.parquet").rename({'Date/Time': 'dt', 'WTG.Name': 'turbine_id'}, axis=1)[:-3]
power_2022 = pd.read_parquet(input_path + "dynamic_report_ewp02_2022_10min.parquet").rename({'Date/Time': 'dt', 'WTG.Name': 'turbine_id'}, axis=1)[:-3]
power = pd.concat([power_2020, power_2021, power_2022], ignore_index=True)

gj_y = pd.read_parquet(input_path + "train_y.parquet").rename({'end_datetime': 'dt'}, axis=1)
ldaps = pd.read_parquet(input_path + "train_ldaps_gyeongju.parquet")

print("Power: ", power.shape)
print("train_y: ", gj_y.shape)
print("LDAPS: ", ldaps.shape)

Power:  (155528, 29)
train_y:  (52608, 4)
LDAPS:  (235818, 15)


In [26]:
datas = [power, gj_y, ldaps]
for d in datas:
    try:
        d['dt'] = (pd.to_datetime(d['dt'])
                    .dt
                    .tz_convert("Asia/Seoul"))
    except TypeError:
        d['dt'] = (pd.to_datetime(d['dt'])
                    .dt
                    .tz_localize("Asia/Seoul"))

gj_y = (gj_y.loc[(gj_y['plant_name'] == "경주풍력")
                 & (gj_y['dt']).between('2020-01-01', '2022-12-31', inclusive='left')])

ldaps = ldaps.loc[ldaps['dt'].between('2020-01-01', '2022-12-31', inclusive='left')]

print("Power: ", power.shape, power['dt'].min(), power['dt'].max())
print("train_y: ", gj_y.shape, gj_y['dt'].min(), gj_y['dt'].max())
print("LDAPS: ", ldaps.shape, ldaps['dt'].min(), ldaps['dt'].max())

Power:  (155528, 29) 2020-01-01 00:00:00+09:00 2022-12-31 23:50:00+09:00
train_y:  (26279, 4) 2020-01-01 01:00:00+09:00 2022-12-30 23:00:00+09:00
LDAPS:  (235602, 15) 2020-01-02 00:00:00+09:00 2022-12-30 23:00:00+09:00


In [27]:
# add wind speed in 100m feature
ldaps["wind_speed"], ldaps["wind_direction"] = uv_to_wsd(
    ldaps["wind_u_10m"], ldaps["wind_v_10m"]
)

ldaps["wind_speed_100m"] = logarithmic_profile(ldaps["wind_speed"], 10, 100, 0.3)

C:\Users\User\AppData\Local\Temp\ipykernel_23468\85188720.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ldaps["wind_speed"], ldaps["wind_direction"] = uv_to_wsd(
C:\Users\User\AppData\Local\Temp\ipykernel_23468\85188720.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ldaps["wind_speed"], ldaps["wind_direction"] = uv_to_wsd(
C:\Users\User\AppData\Local\Temp\ipykernel_23468\85188720.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_i

In [28]:
print(ldaps.columns)
print(power.columns)

Index(['dt', 'elevation', 'land_cover', 'surf_rough', 'frictional_vmax_50m',
       'frictional_vmin_50m', 'pressure', 'relative_humid', 'specific_humid',
       'temp_air', 'storm_u_5m', 'storm_v_5m', 'wind_u_10m', 'wind_v_10m',
       'turbine_id', 'wind_speed', 'wind_direction', 'wind_speed_100m'],
      dtype='object')
Index(['dt', 'turbine_id', 'WTG.Serial', 'AvailabilityForcedOutageTime[Min.]',
       'AvailabilityFullPerformanceTime[Min.]',
       'AvailabilityRequestedShutdownTime[Min.]',
       'AvailabilityScheduledMaintenanceTime[Min.]',
       'AvailabilityTechnicalStandbyTime[Min.]',
       'EnergyProductionActiveEnergyProduction[KWh]',
       'GeneratorAverageWindingTemp.[℃]', 'GridActivePower[kW]',
       'GridReactivePower[kVAr]', 'HydraulicSystemPressure[bar]',
       'NacelleAirDensity[kg/㎥]', 'NacelleNacellePosition[deg]',
       'NacelleOutdoorTemp[℃]', 'NacelleWindDirection[deg]',
       'NacelleWindSpeed[m/s]', 'RotorBlade1Pos.[deg]', 'RotorBlade2Pos.[deg]',
     

In [29]:

power.interpolate(method='linear', inplace=True)

C:\Users\User\AppData\Local\Temp\ipykernel_23468\2284999207.py:1: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  power.interpolate(method='linear', inplace=True)


In [30]:
power['EnergyProductionActiveEnergyProduction[KWh]'].sum()

17927568.0

In [32]:
power_h = power[['dt', 'EnergyProductionActiveEnergyProduction[KWh]']]

power_h = power.resample('H', on='dt').sum() # tlr

# print(df_hourly.head())

C:\Users\User\AppData\Local\Temp\ipykernel_23468\1740558130.py:3: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  power_h = power.resample('H', on='dt').sum()


In [34]:
power_h['EnergyProductionActiveEnergyProduction[KWh]'].sum() # 시간당 발전량의 발전소 단위 평균과 얼추 비슷한 단위가 나옴. 
# 그러므로 sum을 label로

17927568.0

In [35]:
wtg1_df = pd.merge(ldaps, power_h, on = ['dt'])


In [36]:
from astral import LocationInfo
from astral.sun import sun
import pytz

location = LocationInfo("Seoul", "South Korea", "Asia/Seoul", 35.72409, 129.3746)

# 밤/낮 feature 추가

def is_day_or_night(dt):
    if dt.tzinfo is not None:
        dt = dt.tz_convert('Asia/Seoul')
    else:
        dt = dt.tz_localize('Asia/Seoul')
    
    s = sun(location.observer, date=dt)
    sunrise = s['sunrise']
    sunset = s['sunset']
    
    if sunrise < dt < sunset:
        return 0  # Day
    else:
        return 1  # Night

wtg1_df['Day_Night'] = wtg1_df['dt'].apply(is_day_or_night)
wtg1_df.head()


,dt,elevation,land_cover,surf_rough,frictional_vmax_50m,frictional_vmin_50m,pressure,relative_humid,specific_humid,temp_air,...,RotorBlade3Pos.[deg],RotorMotor1Pos.[deg],RotorMotor2Pos.[deg],RotorMotor3Pos.[deg],RotorPitch1Angle[deg],RotorPitch2Angle[deg],RotorPitch3Angle[deg],RotorRotorSpeed[rpm],YawYawcablewindup[deg],Day_Night
0,2020-01-02 00:00:00+09:00,387.640625,1.0,0.286911,10.428498,10.059580,97974.593750,91.796478,0.002686,269.467560,...,88.277039,87.729630,87.727097,87.727165,87.728065,87.730698,87.728134,80.915436,2299.693115,1
1,2020-01-02 01:00:00+09:00,387.640625,1.0,0.286911,10.472921,10.044404,97970.132812,91.729774,0.002686,269.430847,...,75.720596,75.119034,75.120232,75.119667,75.118935,75.121033,75.119667,80.925980,2304.814941,1
2,2020-01-02 02:00:00+09:00,387.640625,1.0,0.286911,10.682985,10.478634,97951.546875,92.788666,0.002686,269.374390,...,75.713768,75.044365,75.045097,75.045631,75.044403,75.045067,75.045631,80.982483,2304.814941,1
3,2020-01-02 03:00:00+09:00,387.640625,1.0,0.286911,10.676681,10.090029,97908.968750,92.494576,0.002686,269.352112,...,91.202034,90.510536,90.510330,90.509796,90.511368,90.511604,90.510834,80.949768,2304.814941,1
4,2020-01-02 04:00:00+09:00,387.640625,1.0,0.286911,10.079557,9.672620,97858.398438,88.986443,0.002686,269.413269,...,48.426533,47.839668,47.839432,47.840401,47.839668,47.839401,47.840401,80.988869,2288.006348,1


In [37]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# wtg1_df의 열 이름에서 특수 문자를 '_'로 교체
wtg1_df.columns = wtg1_df.columns.str.replace(r'[<>\[\]]', '_', regex=True)
wtg1_df.columns = wtg1_df.columns.str.replace(r'[^\w]', '_', regex=True)
wtg1_df.columns = wtg1_df.columns.str.replace(r'__+', '_', regex=True)  # '__'를 '_'로 변경


In [38]:
wtg1_df.columns

Index(['dt', 'elevation', 'land_cover', 'surf_rough', 'frictional_vmax_50m',
       'frictional_vmin_50m', 'pressure', 'relative_humid', 'specific_humid',
       'temp_air', 'storm_u_5m', 'storm_v_5m', 'wind_u_10m', 'wind_v_10m',
       'turbine_id_x', 'wind_speed', 'wind_direction', 'wind_speed_100m',
       'turbine_id_y', 'WTG_Serial', 'AvailabilityForcedOutageTime_Min_',
       'AvailabilityFullPerformanceTime_Min_',
       'AvailabilityRequestedShutdownTime_Min_',
       'AvailabilityScheduledMaintenanceTime_Min_',
       'AvailabilityTechnicalStandbyTime_Min_',
       'EnergyProductionActiveEnergyProduction_KWh_',
       'GeneratorAverageWindingTemp_', 'GridActivePower_kW_',
       'GridReactivePower_kVAr_', 'HydraulicSystemPressure_bar_',
       'NacelleAirDensity_kg_', 'NacelleNacellePosition_deg_',
       'NacelleOutdoorTemp_', 'NacelleWindDirection_deg_',
       'NacelleWindSpeed_m_s_', 'RotorBlade1Pos_deg_', 'RotorBlade2Pos_deg_',
       'RotorBlade3Pos_deg_', 'RotorMotor1Po

In [39]:
target = 'EnergyProductionActiveEnergyProduction_KWh_'

In [47]:
numeric_columns = wtg1_df.select_dtypes(include=['float', 'int']).columns.drop(target)


In [49]:
x_train = wtg1_df.loc[wtg1_df['dt'].between('2020-01-01', '2020-12-31', inclusive='left'), numeric_columns]
x_test = wtg1_df.loc[wtg1_df['dt'].between('2021-01-01', '2022-12-31', inclusive='left'), numeric_columns]

y_train = wtg1_df.loc[wtg1_df['dt'].between('2020-01-01', '2020-12-31', inclusive='left'), target].shift(-144)
y_test = wtg1_df.loc[wtg1_df['dt'].between('2021-01-01', '2022-12-31', inclusive='left'), target].shift(-144)

x_train = x_train.iloc[:-144]
y_train = y_train.iloc[:-144]

x_test = x_test.iloc[:-144]
y_test = y_test.iloc[:-144]


In [64]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

def get_trasforms_datas():
    z_scaler = StandardScaler()
    minmax_scaler = MinMaxScaler()
    
    x_train = wtg1_df.loc[wtg1_df['dt'].between('2020-01-01', '2020-12-31', inclusive='left'), numeric_columns]
    x_test = wtg1_df.loc[wtg1_df['dt'].between('2021-01-01', '2022-12-31', inclusive='left'), numeric_columns]

    y_train = wtg1_df.loc[wtg1_df['dt'].between('2020-01-01', '2020-12-31', inclusive='left'), target].shift(-144)
    y_test = wtg1_df.loc[wtg1_df['dt'].between('2021-01-01', '2022-12-31', inclusive='left'), target].shift(-144)

    x_train = x_train.iloc[:-144]
    y_train = y_train.iloc[:-144]

    x_test = x_test.iloc[:-144]
    y_test = y_test.iloc[:-144]

    # Min-Max Scaling
    x_train_m = minmax_scaler.fit_transform(x_train)
    x_train_m = pd.DataFrame(x_train_m, columns=x_train.columns)
    x_test_m = minmax_scaler.transform(x_test)
    x_test_m = pd.DataFrame(x_test_m, columns=x_train.columns)

    # Standard Scaling
    x_train_z = z_scaler.fit_transform(x_train)
    x_train_z = pd.DataFrame(x_train_z, columns=x_train.columns)
    x_test_z = z_scaler.transform(x_test)
    x_test_z = pd.DataFrame(x_test_z, columns=x_train.columns)

    return x_train, x_test, x_train_m, x_test_m, x_train_z, x_test_z, y_train, y_test


In [65]:
x_train, x_test, x_train_m, x_test_m, x_train_z, x_test_z, y_train, y_test = get_trasforms_datas()


In [71]:
from sklearn.cluster import KMeans

# 이제 특징 생성에 클러스터링 결과는 보지 않을 예정.
def addKmeansFeature(train_data, test_data):
    pd.options.mode.chained_assignment = None

    for n_clusters in range(2, 7):  # 2부터 6까지 클러스터 생성
        kmeans = KMeans(n_clusters=n_clusters, n_init=10)

        train_data[f'cluster_{n_clusters}'] = kmeans.fit_predict(train_data[['wind_speed', 'wind_direction']])
        
        test_data[f'cluster_{n_clusters}'] = kmeans.predict(test_data[['wind_speed', 'wind_direction']])

    return train_data, test_data


In [72]:
from sklearn.decomposition import PCA

def addPCAFeature(train_data, test_data):
    # PCA 적용할 특징 열 선택 (u, v 성분)
    wind_features = ['storm_u_5m', 'storm_v_5m', 'wind_u_10m', 'wind_v_10m', 
                     'wind_speed', 'wind_direction']
    
    # 훈련 데이터에서 PCA 학습
    pca = PCA(n_components=2)
    pca_train = pca.fit_transform(train_data[wind_features])
    
    # 훈련 데이터에 주성분 추가
    train_data['PC1'] = pca_train[:, 0]
    train_data['PC2'] = pca_train[:, 1]
    
    # 테스트 데이터에 PCA 적용
    pca_test = pca.transform(test_data[wind_features])
    test_data['PC1'] = pca_test[:, 0]
    test_data['PC2'] = pca_test[:, 1]

    # PCA 설명력 확인
    explained_variance = pca.explained_variance_ratio_
    print(f"PC1 설명력: {explained_variance[0]}")
    print(f"PC2 설명력: {explained_variance[1]}")

    return train_data, test_data


In [76]:
from sklearn_extra.cluster import KMedoids

def addKMedoidsFeature(train_data, test_data):
    pd.options.mode.chained_assignment = None

    for n_clusters in range(2, 7):  # 2부터 6까지 클러스터 생성
        kmedoids = KMedoids(n_clusters=n_clusters, random_state=42)

        # 훈련 데이터에 K-Medoids 클러스터링 적용
        train_data[f'medoid_cluster_{n_clusters}'] = kmedoids.fit_predict(train_data[['wind_speed', 'wind_direction']])

        # 테스트 데이터에 학습된 K-Medoids 모델 적용
        test_data[f'medoid_cluster_{n_clusters}'] = kmedoids.predict(test_data[['wind_speed', 'wind_direction']])

    return train_data, test_data


In [73]:
x_train.drop(columns=['cluster'], inplace=True, errors='ignore')
x_test.drop(columns=['cluster'], inplace=True, errors='ignore')
x_train_m.drop(columns=['cluster'], inplace=True, errors='ignore')
x_test_m.drop(columns=['cluster'], inplace=True, errors='ignore')
x_train_z.drop(columns=['cluster'], inplace=True, errors='ignore')
x_test_z.drop(columns=['cluster'], inplace=True, errors='ignore')

In [74]:
x_train, x_test = addKmeansFeature(x_train, x_test)
x_train_m, x_test_m = addKmeansFeature(x_train_m, x_test_m)
x_train_z, x_test_z = addKmeansFeature(x_train_z, x_test_z)


In [75]:
# 각 데이터프레임에 addPCAFeature 적용
x_train, x_test = addPCAFeature(x_train, x_test)
x_train_m, x_test_m = addPCAFeature(x_train_m, x_test_m)
x_train_z, x_test_z = addPCAFeature(x_train_z, x_test_z)


PC1 설명력: 0.9982893179797387
PC2 설명력: 0.000787891560487553
PC1 설명력: 0.769956023089617
PC2 설명력: 0.1072706241482032
PC1 설명력: 0.331150624315771
PC2 설명력: 0.3263920177870679


In [77]:
# 각 데이터프레임에 addKMedoidsFeature 적용
x_train, x_test = addKMedoidsFeature(x_train, x_test)
x_train_m, x_test_m = addKMedoidsFeature(x_train_m, x_test_m)
x_train_z, x_test_z = addKMedoidsFeature(x_train_z, x_test_z)


MemoryError: Unable to allocate 4.59 GiB for an array with shape (24816, 24816) and data type float64

In [78]:
x_train.columns

Index(['elevation', 'land_cover', 'surf_rough', 'frictional_vmax_50m',
       'frictional_vmin_50m', 'pressure', 'relative_humid', 'specific_humid',
       'temp_air', 'storm_u_5m', 'storm_v_5m', 'wind_u_10m', 'wind_v_10m',
       'wind_speed', 'wind_direction', 'wind_speed_100m',
       'AvailabilityForcedOutageTime_Min_',
       'AvailabilityFullPerformanceTime_Min_',
       'AvailabilityRequestedShutdownTime_Min_',
       'AvailabilityScheduledMaintenanceTime_Min_',
       'AvailabilityTechnicalStandbyTime_Min_', 'GeneratorAverageWindingTemp_',
       'GridActivePower_kW_', 'GridReactivePower_kVAr_',
       'HydraulicSystemPressure_bar_', 'NacelleAirDensity_kg_',
       'NacelleNacellePosition_deg_', 'NacelleOutdoorTemp_',
       'NacelleWindDirection_deg_', 'NacelleWindSpeed_m_s_',
       'RotorBlade1Pos_deg_', 'RotorBlade2Pos_deg_', 'RotorBlade3Pos_deg_',
       'RotorMotor1Pos_deg_', 'RotorMotor2Pos_deg_', 'RotorMotor3Pos_deg_',
       'RotorPitch1Angle_deg_', 'RotorPitch2Angle_d

In [79]:
x_dict = {
    # 원본 데이터만 사용
    'original': ['elevation', 'land_cover', 'surf_rough', 'frictional_vmax_50m', 'frictional_vmin_50m', 
                 'pressure', 'relative_humid', 'specific_humid', 'temp_air', 'storm_u_5m', 'storm_v_5m', 
                 'wind_u_10m', 'wind_v_10m', 'wind_speed', 'wind_direction', 'wind_speed_100m'],

    # PCA 추가
    'pca_only': ['elevation', 'land_cover', 'surf_rough', 'frictional_vmax_50m', 'frictional_vmin_50m', 
                 'pressure', 'relative_humid', 'specific_humid', 'temp_air', 'storm_u_5m', 'storm_v_5m', 
                 'wind_u_10m', 'wind_v_10m', 'wind_speed', 'wind_direction', 'wind_speed_100m', 'PC1', 'PC2'],

    # 클러스터(2~6) 추가
    'cluster_only': ['elevation', 'land_cover', 'surf_rough', 'frictional_vmax_50m', 'frictional_vmin_50m', 
                     'pressure', 'relative_humid', 'specific_humid', 'temp_air', 'storm_u_5m', 'storm_v_5m', 
                     'wind_u_10m', 'wind_v_10m', 'wind_speed', 'wind_direction', 'wind_speed_100m', 
                     'cluster_2', 'cluster_3', 'cluster_4', 'cluster_5', 'cluster_6'],

    # PCA + 클러스터(2~6) 추가
    'pca_and_cluster': ['elevation', 'land_cover', 'surf_rough', 'frictional_vmax_50m', 'frictional_vmin_50m', 
                        'pressure', 'relative_humid', 'specific_humid', 'temp_air', 'storm_u_5m', 'storm_v_5m', 
                        'wind_u_10m', 'wind_v_10m', 'wind_speed', 'wind_direction', 'wind_speed_100m', 
                        'PC1', 'PC2', 'cluster_2', 'cluster_3', 'cluster_4', 'cluster_5', 'cluster_6'],

    # 클러스터 개수에 따른 경우
    'cluster_2_only': ['elevation', 'land_cover', 'surf_rough', 'frictional_vmax_50m', 'frictional_vmin_50m', 
                       'pressure', 'relative_humid', 'specific_humid', 'temp_air', 'storm_u_5m', 'storm_v_5m', 
                       'wind_u_10m', 'wind_v_10m', 'wind_speed', 'wind_direction', 'wind_speed_100m', 'cluster_2'],
    
    'cluster_3_only': ['elevation', 'land_cover', 'surf_rough', 'frictional_vmax_50m', 'frictional_vmin_50m', 
                       'pressure', 'relative_humid', 'specific_humid', 'temp_air', 'storm_u_5m', 'storm_v_5m', 
                       'wind_u_10m', 'wind_v_10m', 'wind_speed', 'wind_direction', 'wind_speed_100m', 'cluster_3'],
    
    'cluster_4_only': ['elevation', 'land_cover', 'surf_rough', 'frictional_vmax_50m', 'frictional_vmin_50m', 
                       'pressure', 'relative_humid', 'specific_humid', 'temp_air', 'storm_u_5m', 'storm_v_5m', 
                       'wind_u_10m', 'wind_v_10m', 'wind_speed', 'wind_direction', 'wind_speed_100m', 'cluster_4'],
    
    'cluster_5_only': ['elevation', 'land_cover', 'surf_rough', 'frictional_vmax_50m', 'frictional_vmin_50m', 
                       'pressure', 'relative_humid', 'specific_humid', 'temp_air', 'storm_u_5m', 'storm_v_5m', 
                       'wind_u_10m', 'wind_v_10m', 'wind_speed', 'wind_direction', 'wind_speed_100m', 'cluster_5'],
    
    'cluster_6_only': ['elevation', 'land_cover', 'surf_rough', 'frictional_vmax_50m', 'frictional_vmin_50m', 
                       'pressure', 'relative_humid', 'specific_humid', 'temp_air', 'storm_u_5m', 'storm_v_5m', 
                       'wind_u_10m', 'wind_v_10m', 'wind_speed', 'wind_direction', 'wind_speed_100m', 'cluster_6']
}


# 아래는 학습관렵 함수 및 Plot 함수

In [81]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, AdaBoostRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor,XGBRFRegressor
from lightgbm import LGBMRegressor
from sklearn.model_selection import RandomizedSearchCV


In [82]:
# 하이퍼 파라미터 그리드
param_grids = {
    'GradientBoostingRegressor': {'n_estimators': [1, 100], 'learning_rate': [0.01, 0.1]},
    'SVR': {'C': [0.1, 1], 'kernel': ['linear']},
    'RandomForestRegressor': {'n_estimators': [1, 200], 'max_depth': [10, 20, None]},
    'LinearRegression': {},  # LinearRegression은 기본 파라미터로 사용
    'ExtraTreesRegressor': {'n_estimators': [11, 200], 'max_depth': [10, 20, None]},
    'AdaBoostRegressor': {'n_estimators': [1, 100], 'learning_rate': [0.01, 0.1]},
    'DecisionTreeRegressor': {'max_depth': [10, 20, None]},
    'XGBRegressor': {'n_estimators': [1, 100], 'learning_rate': [0.01, 0.1]},
    'XGBRFRegressor': {'n_estimators': [1, 100], 'learning_rate': [0.01, 0.1]},
    'LGBMRegressor' : {'n_estimators': [1, 100], 'learning_rate': [0.01, 0.1]}
}

In [83]:
def plot_feature_importance(model, x_col,x_shape, model_name, save_dir):
    save_dir = os.path.join(ipynb_path, save_dir)
    
    try:
        importances = model.feature_importances_
        indices = np.argsort(importances)[::-1]

        plt.figure(figsize=(10, 6))
        plt.title(f"Feature Importance : {model_name}")
        plt.bar(range(x_shape[1]), importances[indices], align='center')
        plt.xticks(range(x_shape[1]), x_col[indices], rotation=90)
        plt.tight_layout()
        plt.savefig(os.path.join(save_dir, f"{model_name}_feature_importance.png"))
        plt.close()
    except AttributeError:
        if hasattr(model, 'coef_'):
            importances = np.abs(model.coef_)
            indices = np.argsort(importances)[::-1]

            plt.figure(figsize=(10, 6))
            plt.title(f"Coefficient : {model_name}")
            plt.bar(range(x_shape[1]), importances[indices], align='center')
            plt.xticks(range(x_shape[1]), x_col[indices], rotation=90)
            plt.tight_layout()
            plt.savefig(os.path.join(save_dir, f"{model_name}_coefficient.png"))
            plt.close()
        else:
            print(f"{model_name} has no feature importances or coefficients.")

In [84]:
def plot_predict_actual(pred, actual ,model_name, save_dir):
    plt.figure(figsize=(10, 6))
    plt.title(f"Feature Importance : {model_name}")
    plt.xticks(rotation=90)
    plt.plot(list(pred), label='pred')
    plt.plot(list(actual), label='Actual')
    plt.legend()
    plt.tight_layout()
    plt.savefig(os.path.join(save_dir, f"{model_name}_predict.png"))
    plt.close()

In [85]:
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import joblib

def train(save_dir, x_train, y_train, x_test, y_test):
    if not os.path.exists(save_dir):
        os.mkdir(save_dir)
    log_file = open(os.path.join(save_dir, "model_training_log.txt"), "w")

    models = [
        GradientBoostingRegressor(),
        # SVR(),
        RandomForestRegressor(),
        LinearRegression(),
        ExtraTreesRegressor(),
        AdaBoostRegressor(),
        DecisionTreeRegressor(),
        XGBRegressor(),
        XGBRFRegressor(),
        LGBMRegressor(verbose=-1),
    ]

    results = {
        'Model_Name': [],
        'R2_score': [],
        'RMSE': [],
        'NMAE': [],
        'Best_Params': [],
        'Predict' : []
    }

    for model in models:
        model_name_current = model.__class__.__name__

        param_grid = param_grids.get(model_name_current, {})

        search = search = RandomizedSearchCV(model, param_distributions=param_grid, cv=3, n_jobs=-1, n_iter=5)
        log_file.write(f"Searching best hyperparameters for {model_name_current}\n")
        search.fit(x_train, y_train)
        best_model = search.best_estimator_
        best_params = search.best_params_

        log_file.write(f"Training {model_name_current}\n")

        y_pred = best_model.predict(x_test)

        r2 = r2_score(y_test, y_pred) * 100
        rmse_value = mean_squared_error(y_test, y_pred, squared=False)
        mae = mean_absolute_error(y_test, y_pred)
        nmae_value = mae / y_test.mean()

        results['Model_Name'].append(model_name_current)
        results['R2_score'].append(r2)
        results['RMSE'].append(rmse_value)
        results['NMAE'].append(nmae_value)
        results['Best_Params'].append(best_params)
        results['Predict'].append(y_pred)

        joblib.dump(search, os.path.join(save_dir, f'{model_name_current}.pkl'))
        
        plot_predict_actual(y_pred, y_test, model_name_current, save_dir)

        log_file.write(f"R2 Score: {r2:.2f}\n")
        log_file.write(f"RMSE: {rmse_value:.2f}\n")
        log_file.write(f"NMAE: {nmae_value:.4f}\n")
        log_file.write(f"Best Hyperparameters: {best_params}\n\n")

        
        # def plot_feature_importance(model, x_col,x_shape, model_name, save_dir):

        plot_feature_importance(best_model, x_train.columns, x_train.shape, model_name_current,save_dir)

    results_df = pd.DataFrame(results)

    results_df.to_csv(os.path.join(save_dir, "model_scores.csv"), index=False)

    log_file.close()

In [86]:
for key in x_dict.keys():
    # 필요한 컬럼만 선택해서 데이터셋 생성
    x_train_selected = x_train[x_dict[key]]
    x_test_selected = x_test[x_dict[key]]
    x_train_m_selected = x_train_m[x_dict[key]]
    x_test_m_selected = x_test_m[x_dict[key]]
    x_train_z_selected = x_train_z[x_dict[key]]
    x_test_z_selected = x_test_z[x_dict[key]]

    # ====================
    # 정규화 없는 데이터
    # ====================
    save_dir = os.path.join(ipynb_path, f'test10/{key}_General_testBaseline/')
    
    train(save_dir, x_train_selected, y_train, x_test_selected, y_test)

    # ====================
    # Min-Max 정규화
    # ====================
    save_dir = os.path.join(ipynb_path, f'test10/{key}_General_m_testBaseline/')
    
    train(save_dir, x_train_m_selected, y_train, x_test_m_selected, y_test)

    # ====================
    # z-정규화
    # ====================
    save_dir = os.path.join(ipynb_path, f'test10/{key}_General_z_testBaseline/')
    
    train(save_dir, x_train_z_selected, y_train, x_test_z_selected, y_test)


C:\Users\User\anaconda3\Lib\site-packages\sklearn\model_selection\_search.py:318: UserWarning: The total space of parameters 4 is smaller than n_iter=5. Running 4 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
C:\Users\User\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
C:\Users\User\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
C:\Users\User\anaconda3\Lib\site-packages\sklearn\model_selection\_search.py:318: UserWarning: The total space of parameters 1 is smaller than n_iter=5. Running 1 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
C:\Use

KeyboardInterrupt: 